In [1]:
## weaves
## smava

getwd()

## load in packages
library(tidyverse)
library(rpart.plot)

library(e1071)
library(MASS)
library(mlbench)
library(gbm)
library(kernlab)
library(RSNNS)

## last loaded, first found methods
library(caret)

## implementation
library(doMC)

registerDoMC(cores = 4)

options(useFancyQuotes = TRUE)

[1] "/misc/build/0/pyeg0/caret/classr1"

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: rpart


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


Loaded gbm 2.1.5


Attaching package: 'kernlab'


The following object is masked from 'package:purrr':

    cross


The following object is masked from 'package:ggplot2':

    alpha


Loading required package: Rcpp

Loading required package: lattice


Attaching package: 'caret'


The following objects are masked from 'package:RSNNS':

    confusionMatrix, train


The following object is masked from 'package:purrr':

    lift


Loading required package: foreach


Attaching package:

In [3]:
load("bak/in/train.rdata")
load("bak/in/test.rdata")

train0 <- data.frame(train) # a local copy.
smava1 <- list()

In [4]:
cols <- colnames(train)
cols <- cols[grep('^x', cols)]
cols <- setdiff(cols, "x3")

smava1$xcols <- c(cols, "interestRate")

In [5]:
## Not very good. Let's try a predictor specific to a bank
## List the banks.
smava1$banks <- unique(train[["bank"]])
length(smava1$banks)
smava1$banks

[1] 30

[1] B14 B4  B25 B3  B1  B8  B18 B21 B6  B15 B16 B2  B12 B27 B7  B30 B9  B22 B28
[20] B17 B26 B23 B11 B20 B19 B13 B29 B24 B5  B10
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

In [6]:
## Random choose a bank to look at
fidx <- function(N0, n=3) sample(1:N0, n, replace = FALSE)

t <- smava1$banks
tag <- t[fidx(length(t))]
smava1$tag <- tag[1:1]

smava1

$xcols
 [1] "x1"           "x2"           "x4"           "x5"           "x6"          
 [6] "x7"           "x8"           "x9"           "x10"          "interestRate"

$banks
 [1] B14 B4  B25 B3  B1  B8  B18 B21 B6  B15 B16 B2  B12 B27 B7  B30 B9  B22 B28
[20] B17 B26 B23 B11 B20 B19 B13 B29 B24 B5  B10
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

$tag
[1] B5
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

In [8]:
## Specific to a bank
## And impute x2
train1 <- train[ train$bank %in% smava1$tag, smava1$xcols]

train1$x2na <- 0
train1[is.na(train1$x2), "x2na"] <- 1

v0 <- train1[train1$x2 & train1$x2na == 0, "x2"]
v0 <- mean(v0[["x2"]])

train1[is.na(train1$x2), "x2"] <- v0

train1 <- train1[!is.na(train1$interestRate),]

dim(train1)
head(train1)

# test1[ is.na(test1$x2), "x2" ] <- smava0$x2impute

[1] 2269   11

x1,x2,x4,x5,x6,x7,x8,x9,x10,interestRate,x2na
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.7568374,44.4027868,5.6768473,5.60014788,ACS,390PS,11.064204,0.29376064,0.3258803,2.882112,0
3.0023928,3.4174193,5.8046899,0.44332639,FBW,XP9Z1,6.147756,0.03732323,0.3353398,5.096094,0
0.4778104,0.6396738,11.3330166,-3.54184762,ACS,H73HB,7.994641,0.42284321,0.3782313,2.106065,0
0.5909763,115.4112001,0.0595889,-4.25708276,FBW,C5A0Z,-3.969879,0.31924884,0.3592664,3.978989,0
0.4418101,8.0387350,2.4638611,0.09603354,ACS,4TLSR,3.348747,0.41179447,0.4124464,3.316749,1
0.4198342,8.0387350,4.1886184,-5.77934633,FBW,9E5T5,-1.334178,0.70731438,0.4873468,3.936071,1


In [ ]:
## Look for a regression and check Rsquared

regressControl  <- trainControl(method="repeatedcv",
                    number = 4,
                    repeats = 5
                    ) 

## R^2 of only 0.3 is not good enough, with x6 is not better.
regress <- train(interestRate ~ x1 + x10 + x4,
           data = train1,
           method  = "lm",
           trControl = regressControl)
regress

In [9]:
## Let's overfit with a multi-layer perceptron and see if I can use x6 and x7.
## Regression model two inaccurate. A simple multi-layer perceptron - a look up table almost.

fControl  <- trainControl(method="repeatedcv",
                    number = 4,
                    repeats = 5
                    )
method = 'mlp'
method = 'mlpWeightDecay'

mi <- getModelInfo(model = method, regex = FALSE)[[1]]
p0 <- data.frame(mi$parameters)
p0

parameter,class,label
<fct>,<fct>,<fct>
size,numeric,#Hidden Units
decay,numeric,Weight Decay


In [10]:
## Try a simple neural network - with 11 hidden layers, you can get R^2 to 0.11 - awful.

fit0 <- train(interestRate ~ x1 + x10 + x7 + x6,
           data = train1,
           method  = method,
           trControl = fControl, tuneGrid = data.frame(size=c(15), decay=c(0.1)))
fit0

In [ ]:
'lssvmRadial'